# Your Title Here

**Name(s)**: Ethan Cao

**Website Link**: (your website link)

## Code

In [115]:
import pandas as pd
import numpy as np
import os

import plotly.express as px
pd.options.plotting.backend = 'plotly'
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder, FunctionTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.compose import ColumnTransformer

### Framing the Problem

In [116]:
def combine_times(date_col_name, time_col_name, new_col_name, df):
    df = df.copy()
    df[new_col_name] = df[date_col_name] + pd.to_timedelta(df[time_col_name].astype(str))
    return df

data = pd.read_excel("outage.xlsx", skiprows=[0,1,2,3,4,6], index_col=1).iloc[:,1:]
data = combine_times("OUTAGE.START.DATE", 'OUTAGE.START.TIME', 'OUTAGE.START.DATETIME', data)
data = combine_times("OUTAGE.RESTORATION.DATE", "OUTAGE.RESTORATION.TIME", "OUTAGE.RESTORATION.DATETIME", data)

In [117]:
data['CAUSE.CATEGORY'].unique()

array(['severe weather', 'intentional attack',
       'system operability disruption', 'equipment failure',
       'public appeal', 'fuel supply emergency', 'islanding'],
      dtype=object)

In [118]:
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_URBAN,POPDEN_UC,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START.DATETIME,OUTAGE.RESTORATION.DATETIME
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,2279.0,1700.5,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00
1531,2006,NaN,North Dakota,ND,MRO,West North Central,NaN,NaN,NaT,NaN,...,2192.2,1868.2,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,2038.3,1905.4,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00


In [119]:
data['IS.HURRICANE'] = data['HURRICANE.NAMES'].isna() == False

In [120]:
data['MONTH.START'] = data['OUTAGE.START.DATETIME'].apply(lambda x : x.month)

In [193]:
data.columns

Index(['YEAR', 'MONTH', 'U.S._STATE', 'POSTAL.CODE', 'NERC.REGION',
       'CLIMATE.REGION', 'ANOMALY.LEVEL', 'CLIMATE.CATEGORY',
       'OUTAGE.START.DATE', 'OUTAGE.START.TIME', 'OUTAGE.RESTORATION.DATE',
       'OUTAGE.RESTORATION.TIME', 'CAUSE.CATEGORY', 'CAUSE.CATEGORY.DETAIL',
       'HURRICANE.NAMES', 'OUTAGE.DURATION', 'DEMAND.LOSS.MW',
       'CUSTOMERS.AFFECTED', 'RES.PRICE', 'COM.PRICE', 'IND.PRICE',
       'TOTAL.PRICE', 'RES.SALES', 'COM.SALES', 'IND.SALES', 'TOTAL.SALES',
       'RES.PERCEN', 'COM.PERCEN', 'IND.PERCEN', 'RES.CUSTOMERS',
       'COM.CUSTOMERS', 'IND.CUSTOMERS', 'TOTAL.CUSTOMERS', 'RES.CUST.PCT',
       'COM.CUST.PCT', 'IND.CUST.PCT', 'PC.REALGSP.STATE', 'PC.REALGSP.USA',
       'PC.REALGSP.REL', 'PC.REALGSP.CHANGE', 'UTIL.REALGSP', 'TOTAL.REALGSP',
       'UTIL.CONTRI', 'PI.UTIL.OFUSA', 'POPULATION', 'POPPCT_URBAN',
       'POPPCT_UC', 'POPDEN_URBAN', 'POPDEN_UC', 'POPDEN_RURAL',
       'AREAPCT_URBAN', 'AREAPCT_UC', 'PCT_LAND', 'PCT_WATER_TOT',
       'PCT

In [122]:
data['IS.HURRICANE'].sum()

72

In [86]:
data['CAUSE.CATEGORY'].value_counts()

CAUSE.CATEGORY
severe weather                   763
intentional attack               418
system operability disruption    127
public appeal                     69
equipment failure                 60
fuel supply emergency             51
islanding                         46
Name: count, dtype: int64

In [87]:
793/1534

0.5169491525423728

In [195]:
features = ['POSTAL.CODE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'MONTH', 'CAUSE.CATEGORY']

In [196]:
data['ANOMALY.LEVEL'] = data['ANOMALY.LEVEL'].fillna(0)

In [217]:
px.bar(data,x='CAUSE.CATEGORY',y='CUSTOMERS.AFFECTED')

### Baseline Model

In [255]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline

from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.compose import ColumnTransformer



data_class = data[features].dropna()

X = data_class[['POSTAL.CODE', 'ANOMALY.LEVEL', 'MONTH']]
y = data_class['CAUSE.CATEGORY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
col_trans = ColumnTransformer([
    ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['POSTAL.CODE'])],
    remainder = 'passthrough')
pl = Pipeline([
    ('col_trans', col_trans),
    ('forest', RandomForestClassifier())
])

pl.fit(X_train, y_train)
prediction = pl.predict(X_test)


In [139]:
accuracy_score(y_test, prediction)

0.8443396226415094

In [257]:
import pprint as pp

### Final Model

In [213]:
features = ['POSTAL.CODE', 'ANOMALY.LEVEL', 'OUTAGE.DURATION', 'MONTH','CUSTOMERS.AFFECTED', 'CAUSE.CATEGORY']
data_class = data[features].dropna()

X = data_class[['POSTAL.CODE', 'ANOMALY.LEVEL', 'MONTH','OUTAGE.DURATION','CUSTOMERS.AFFECTED']]
y = data_class['CAUSE.CATEGORY']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
col_trans = ColumnTransformer([
    ('one-hot', OneHotEncoder(handle_unknown='ignore'), ['POSTAL.CODE'])],
    remainder = 'passthrough')
pl = Pipeline([
    ('col_trans', col_trans),
    ('forest', RandomForestClassifier(max_depth = 24))
])

pl.fit(X_train, y_train)
prediction = pl.predict(X_test)


In [96]:
param_grid = {"forest__max_depth": np.append(np.arange(5,30),None)}
search = GridSearchCV(pl, param_grid,cv=5)
search.fit(X_train,y_train)

/Users/billwang/anaconda3/envs/DSC80/lib/python3.8/site-packages/sklearn/model_selection/_split.py:676: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=5.
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('col_trans',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('one-hot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['POSTAL.CODE'])])),
                                       ('forest', RandomForestClassifier())]),
             param_grid={'forest__max_depth': array([5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, None], dtype=object)})

In [97]:
search.best_params_

{'forest__max_depth': 24}

In [214]:
prediction

array(['severe weather', 'severe weather', 'severe weather',
       'severe weather', 'intentional attack', 'severe weather',
       'severe weather', 'public appeal', 'severe weather',
       'severe weather', 'intentional attack', 'public appeal',
       'intentional attack', 'severe weather', 'intentional attack',
       'severe weather', 'severe weather', 'severe weather',
       'severe weather', 'severe weather', 'intentional attack',
       'intentional attack', 'severe weather', 'intentional attack',
       'severe weather', 'severe weather', 'severe weather',
       'severe weather', 'severe weather', 'severe weather',
       'severe weather', 'islanding', 'severe weather',
       'system operability disruption', 'public appeal',
       'intentional attack', 'severe weather', 'severe weather',
       'severe weather', 'severe weather', 'severe weather',
       'severe weather', 'severe weather', 'severe weather',
       'severe weather', 'system operability disruption',
      

In [215]:
search.cv_results_['mean_test_score']

array([0.81161595, 0.82345731, 0.83057199, 0.84596365, 0.85426176,
       0.85902367, 0.85783319, 0.86258101, 0.86613835, 0.86376444,
       0.86730769, 0.8602071 , 0.86612426, 0.86612426, 0.86494787,
       0.86850521, 0.86968864, 0.86731474, 0.86730769, 0.87323894,
       0.86256692, 0.86613835, 0.86493378, 0.86494083, 0.86730065,
       0.86494787])

In [216]:
accuracy_score(y_test, prediction)

0.8490566037735849

In [217]:
pl.score(X_train, y_train)

0.9988151658767772

### Fairness Analysis

In [218]:
check_diff_class = X_test.assign(prediction = prediction).assign(actual = y_test)
check_diff_class['Predict_right'] = check_diff_class['prediction'] == check_diff_class['actual']
check_diff_class

,POSTAL.CODE,ANOMALY.LEVEL,MONTH,OUTAGE.DURATION,CUSTOMERS.AFFECTED,prediction,actual,Predict_right
OBS,,,,,,,,
524,MD,0.7,12.0,2279.0,122000.0,severe weather,severe weather,True
316,IN,0.7,1.0,15950.0,114791.0,severe weather,severe weather,True
355,IL,0.1,7.0,420.0,181000.0,severe weather,severe weather,True
1107,CA,0.9,12.0,8472.0,850068.0,severe weather,severe weather,True
1450,MA,-0.4,2.0,2891.0,1.0,intentional attack,fuel supply emergency,False
...,...,...,...,...,...,...,...,...
1440,MA,-0.4,2.0,4745.0,50000.0,severe weather,severe weather,True
1435,MA,-0.1,6.0,31.0,29250.0,system operability disruption,system operability disruption,True
1115,CA,-0.5,6.0,224.0,477.0,islanding,severe weather,False


In [219]:
check_diff_class['actual'].unique()

array(['severe weather', 'fuel supply emergency', 'public appeal',
       'islanding', 'intentional attack', 'system operability disruption',
       'equipment failure'], dtype=object)

In [220]:
data

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START.DATETIME,OUTAGE.RESTORATION.DATETIME,IS.HURRICANE,MONTH.START
OBS,,,,,,,,,,,,,,,,,,,,,
1,2011,7.0,Minnesota,MN,MRO,East North Central,-0.3,normal,2011-07-01,17:00:00,...,18.2,2.14,0.60,91.592666,8.407334,5.478743,2011-07-01 17:00:00,2011-07-03 20:00:00,False,7.0
2,2014,5.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2014-05-11,18:38:00,...,18.2,2.14,0.60,91.592666,8.407334,5.478743,2014-05-11 18:38:00,2014-05-11 18:39:00,False,5.0
3,2010,10.0,Minnesota,MN,MRO,East North Central,-1.5,cold,2010-10-26,20:00:00,...,18.2,2.14,0.60,91.592666,8.407334,5.478743,2010-10-26 20:00:00,2010-10-28 22:00:00,False,10.0
4,2012,6.0,Minnesota,MN,MRO,East North Central,-0.1,normal,2012-06-19,04:30:00,...,18.2,2.14,0.60,91.592666,8.407334,5.478743,2012-06-19 04:30:00,2012-06-20 23:00:00,False,6.0
5,2015,7.0,Minnesota,MN,MRO,East North Central,1.2,warm,2015-07-18,02:00:00,...,18.2,2.14,0.60,91.592666,8.407334,5.478743,2015-07-18 02:00:00,2015-07-19 07:00:00,False,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1530,2011,12.0,North Dakota,ND,MRO,West North Central,-0.9,cold,2011-12-06,08:00:00,...,3.9,0.27,0.10,97.599649,2.401765,2.401765,2011-12-06 08:00:00,2011-12-06 20:00:00,False,12.0
1531,2006,NaN,North Dakota,ND,MRO,West North Central,0.0,NaN,NaT,NaN,...,3.9,0.27,0.10,97.599649,2.401765,2.401765,NaT,NaT,False,NaN
1532,2009,8.0,South Dakota,SD,RFC,West North Central,0.5,warm,2009-08-29,22:54:00,...,4.7,0.30,0.15,98.307744,1.692256,1.692256,2009-08-29 22:54:00,2009-08-29 23:53:00,False,8.0


In [221]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
import plotly
group_accuracies = check_diff_class[['prediction', 'actual', 'Predict_right']].groupby('actual')['Predict_right'].mean().sort_values(ascending=False)

# Plotting the accuracy for each group as a bar plot
fig = px.bar(x=group_accuracies.index, y=group_accuracies.values,
             labels={'x': 'Postal Code', 'y': 'Accuracy'},
             title='Accuracy by Postal Code Groups')

fig.update_traces(marker_color='rgb(0,0,140)')  # Change bar color if desired
fig.update_xaxes(tickangle=45)
fig.show()

In [228]:
fig.write_html('accuracy_bar.html', include_plotlyjs='cdn')

In [229]:
data[data['CAUSE.CATEGORY'] == 'fuel supply emergency']

,YEAR,MONTH,U.S._STATE,POSTAL.CODE,NERC.REGION,CLIMATE.REGION,ANOMALY.LEVEL,CLIMATE.CATEGORY,OUTAGE.START.DATE,OUTAGE.START.TIME,...,POPDEN_RURAL,AREAPCT_URBAN,AREAPCT_UC,PCT_LAND,PCT_WATER_TOT,PCT_WATER_INLAND,OUTAGE.START.DATETIME,OUTAGE.RESTORATION.DATETIME,IS.HURRICANE,MONTH.START
OBS,,,,,,,,,,,,,,,,,,,,,
50,2014,6.0,Wisconsin,WI,MRO,East North Central,0.0,normal,2014-06-27,13:21:00,...,32.5,3.47,0.90,82.689019,17.312508,3.049041,2014-06-27 13:21:00,NaT,False,6.0
54,2014,1.0,Wisconsin,WI,RFC,East North Central,-0.5,cold,2014-01-24,00:00:00,...,32.5,3.47,0.90,82.689019,17.312508,3.049041,2014-01-24 00:00:00,2014-04-09 11:53:00,False,1.0
55,2014,4.0,Wisconsin,WI,MRO,East North Central,-0.2,normal,2014-04-03,14:45:00,...,32.5,3.47,0.90,82.689019,17.312508,3.049041,2014-04-03 14:45:00,2014-04-09 11:53:00,False,4.0
64,2014,3.0,Wisconsin,WI,MRO,East North Central,-0.4,normal,2014-03-04,09:06:00,...,32.5,3.47,0.90,82.689019,17.312508,3.049041,2014-03-04 09:06:00,2014-03-17 09:06:00,False,3.0
66,2013,8.0,Wisconsin,WI,MRO,East North Central,-0.3,normal,2013-08-07,07:30:00,...,32.5,3.47,0.90,82.689019,17.312508,3.049041,2013-08-07 07:30:00,2013-08-07 09:14:00,False,8.0
183,2007,9.0,Texas,TX,WECC,South,-0.9,cold,2007-09-06,20:00:00,...,15.2,3.35,0.58,97.258336,2.742036,2.090873,2007-09-06 20:00:00,NaT,False,9.0
193,2014,4.0,Texas,TX,TRE,South,-0.2,normal,2014-04-03,00:00:00,...,15.2,3.35,0.58,97.258336,2.742036,2.090873,2014-04-03 00:00:00,NaT,False,4.0
220,2013,12.0,Texas,TX,TRE,South,-0.3,normal,2013-12-13,11:00:00,...,15.2,3.35,0.58,97.258336,2.742036,2.090873,2013-12-13 11:00:00,2013-12-27 11:00:00,False,12.0
221,2012,11.0,Texas,TX,TRE,South,0.1,normal,2012-11-17,10:00:00,...,15.2,3.35,0.58,97.258336,2.742036,2.090873,2012-11-17 10:00:00,2012-11-18 10:00:00,False,11.0


In [222]:
cm

array([[  0,   0,   0,   0,   0,   0,   1],
       [  0,   0,   0,   0,   0,   0,   0],
       [  0,   1,  34,   0,   2,   1,   0],
       [  0,   0,   0,   3,   0,   1,   0],
       [  0,   0,   0,   0,   4,   0,   0],
       [  1,   0,   1,   1,   2, 141,   5],
       [  0,   0,   0,   0,   0,   4,  10]])

In [223]:
# 0 equipment failure                0.000000
# 1 fuel supply emergency            0.000000
# 2 intentional attack               1.000000
# 3 islanding                        0.600000
# 4 public appeal                    0.571429
# 5 severe weather                   0.970803
# 6 system operability disruption    0.217391


In [224]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

# Replace 'y_test' and 'prediction' with your actual test labels and predictions
# Example data (replace this with your actual data)


# Replace these labels with your specific category names
category_labels = [
    'equipment failure',
    'fuel supply emergency',
    'intentional attack',
    'islanding',
    'public appeal',
    'severe weather',
    'system operability disruption'
]


# Calculate the confusion matrix
cm = confusion_matrix(check_diff_class['prediction'], check_diff_class['actual'])

# Create a Plotly heatmap for the confusion matrix
fig = ff.create_annotated_heatmap(z=cm, x=category_labels, y=category_labels, colorscale='Blues')

# Update the layout
fig.update_layout(title='Confusion Matrix',
                  xaxis=dict(title='Predicted Label'),
                  yaxis=dict(title='True Label'))

# Display the confusion matrix
fig.show()
fig.write_html('confusion_matrix.html', include_plotlyjs='cdn')


In [225]:
from sklearn.metrics import precision_score, recall_score

# Assuming 'y_test' contains the true labels and 'prediction' contains predicted labels
precision = precision_score(y_test, prediction, average='weighted')
recall = recall_score(y_test, prediction, average='weighted')

In [226]:
precision

0.833396271640225

In [227]:
recall

0.8490566037735849

In [ ]:
for i in